# Notebook which models the Beam Profile of LED's using a Genetic Algorithm

Module created called "CPM_Genetic_Algorithm.py" which contains all functions below and can be imported into otrher notebooks

In [3]:
from ipywidgets import interactive
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import csv , warnings
import ipywidgets as widgets
from ipywidgets import HBox
import random
import operator
import pandas as pd

Import relevant python libraries

In [4]:
def I_calc_cos2(x , c1 , c2 , c3) :
    Power = []
    c2_rad = np.array(c2)*np.pi/180 #convert to radians as numpy takes radians    
    for item in x :
        I = 0
        for i in range(min(len(c1),2)) :
            I_temp = 0 
            I_temp = c1[i]*(np.cos(abs(item) - c2_rad[i])**c3[i])
            I = I + I_temp
            
        if len(c1) == 3 :
            I_temp = 0 
            I_temp = c1[2]*(np.cos(abs(item) - c2_rad[2])**c3[2])
            I = I - I_temp            
            
        Power.append(I)
        
    return Power

Function which completes model calculation as per the equation : 

$I(\theta) = \sum_i c1_i cos(|\theta| - c2_i)^{c3_i} $

In [5]:
def RMSE(absError , yData) : 
    SE = np.square(absError) # squared errors
    MSE = np.mean(SE) # mean squared errors
    RMSE = np.sqrt(MSE) # Root Mean Squared Error, RMSE
    
    return RMSE

Calculates Root Mean Square Error given the absolute error and target values. Used to calculate the fitness of the model

In [6]:
class Params : 
    
    def __init__(self , c1 , c2 , c3) :
        self.c1 = c1 
        self.c2 = c2
        self.c3 = c3
        
    def distance(self) :
        profile = I_calc_cos2(x_axis_rad , self.c1 , self.c2 , self.c3)
        
        absError = abs(np.array(bprofile) - np.array(profile))
        distance = RMSE(absError , bprofile)
        
        return distance
    
    def __repr__(self):
        return "(" + str(self.c1) + "," + str(self.c2) +  "," + str(self.c3) + ")"

Python Class used to set up the relevant parameters ascretaining to the model. Params.distance() calculates the "distance" betweewn the modeled beam profile and the actual profile. The distance is the RMSE of the model.

In [7]:
class Fitness :
    def __init__(self , profile) :
        self.profile = profile
        self.distance = 0
        self.fitness = 0.0     
        
    def profiledistance(self) :
        if self.distance == 0 :
            beam_profile = self.profile
            pathdistance = beam_profile.distance()
            
        self.distance = pathdistance
        
        return self.distance
       
    def profilefitness(self) :
        if self.fitness == 0 :
            self.fitness = 1 / float(self.profiledistance())
            
        return self.fitness

The fitness flips the RMSE value on its head. The fitness of a Genetic Algorithm looks to promote the fittest chromosones. For this problem we are looking to minimize the RMSE of the model hence we are maximizing 1/RMSE which equates to the fitness of the model

In [8]:
def initialPopulation(popSize):
    population = []

    for i in range(0, popSize):
        population.append(Params(c1= [1, float(random.random()) , float(random.random()) ] , 
                                 c2= [0, int(random.random() * 90) , int(random.random() * 90)] ,
                                 c3= [int(random.random()*10) , int(random.random()*200) , int(random.random()*200) ]))
    return population

The initial population is generated by the population size. We know that c1 corresponds to the maximum intensity of the cosine wave. c2 corresponds to the angle between peaks and c3 corresponds to the width. The best genetic algorithm models have confined boundaries. Given that most beam profiles consist of a single large peak, the first parameter of c1 is 1 while the angle is close to 0 and the width is left variable. 

In [9]:
def rankParams(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).profilefitness(bprofile , x_axis_rad)
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

Ranks the models based on their fitness values. Parameter values are stored using the Params class

In [9]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

The fittest parameters are automatically selected up to and including the length of the elitesize variable. The other values are selected randomly from the group

In [10]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

This function selects all selected parameters from the selection function.

In [11]:
def breed(parent1, parent2):
    child = []

    geneA = random.randint(1,3)
    if geneA == 1 :
        child.append(Params(parent1.c1 , parent2.c2 , parent2.c3))
    elif geneA == 2 :
        child.append(Params(parent2.c1 , parent1.c2 , parent2.c3))
    else :
        child.append(Params(parent2.c1 , parent2.c2 , parent1.c3))
    return child

This funciton is used to crossover two chormosones together based on a random integer between 1 and 3. Crossover occurs between the c1 , c2 or c3 parameter of parent1 and the other two parameters of parent 2. e.g. if the random integer is 2 , the new chromosone would consist of c1 and c3 from parent 2 and c2 from parent2

In [12]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child[0])
    return children

Ensures the fittest parameters are automatically selected and then performs crossover between the ith and ith+1 element starting from the fittest and ending at the length of the population minus the elitesize

In [13]:
def mutate(individual, mutationRate):

    if(random.random() < mutationRate):
        chance = random.randint(1,3)
        if chance == 1 :
            individual.c1 = [1 , float(random.random()), float(random.random())] 
        elif chance == 2 :
            individual.c2 = [0 , int(random.random() * 90) , int(random.random() * 90)]
        else :
            individual.c3 = [int(random.random()*10) , int(random.random()*200) ,int(random.random()*200) ]
            
    return individual

Introduces mutatation into the population if a sample is lower than the mutation rate

In [14]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

Executes the mutate function for the entire set of the population

In [15]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankParams(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

Executes the above funcitons so that they can be performed iteratively without the need for excess coding

In [16]:
def geneticAlgorithm(popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize)
    progress = []
    err = 1 / rankParams(pop)[0][1]
    progress.append(err)
    print("Initial Error: " + str(err*100)[:5] + " %")
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankParams(pop)[0][1])
    
    print("Final Error: " + str((1 / rankParams(pop)[0][1])*100)[:5] + " %")
    bestprofileIndex = rankParams(pop)[0][0]
    bestprofile = pop[bestprofileIndex]
    return bestprofile , progress

Ties all the previous funcitons into one package and prints stating and finishing error as well as keeping note of the progress

In [42]:
def model_param(ParameterTuple) :
    c1 = ParameterTuple.c1
    c2 = ParameterTuple.c2
    c3 = ParameterTuple.c3
        
    return c1 , c2 , c3 

Removes the model parameters from the class so that they are in a user friendly format